<a href="https://colab.research.google.com/github/DAbbott93/Dean-Abbott--Dissertation/blob/main/Model_4_Bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Hope Speech Classification

Model 4: BERT


Installations

In [5]:
!pip install --upgrade transformers

  Using cached transformers-4.10.0-py3-none-any.whl (2.8 MB)
  Using cached tokenizers-0.10.3-cp37-cp37m-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_12_x86_64.manylinux2010_x86_64.whl (3.3 MB)
  Using cached PyYAML-5.4.1-cp37-cp37m-manylinux1_x86_64.whl (636 kB)
  Using cached sacremoses-0.0.45-py3-none-any.whl (895 kB)
  Using cached huggingface_hub-0.0.16-py3-none-any.whl (50 kB)
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [6]:
!pip install simpletransformers

  Using cached simpletransformers-0.61.13-py3-none-any.whl (221 kB)
  Using cached seqeval-1.2.2-py3-none-any.whl
  Using cached wandb-0.12.1-py2.py3-none-any.whl (1.7 MB)
  Using cached tensorboardX-2.4-py2.py3-none-any.whl (124 kB)
  Using cached streamlit-0.87.0-py2.py3-none-any.whl (8.0 MB)
  Using cached datasets-1.11.0-py3-none-any.whl (264 kB)
  Using cached sentencepiece-0.1.96-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.2 MB)
  Using cached GitPython-3.1.18-py3-none-any.whl (170 kB)
  Using cached configparser-5.0.2-py3-none-any.whl (19 kB)
  Using cached sentry_sdk-1.3.1-py2.py3-none-any.whl (133 kB)
  Using cached docker_pycreds-0.4.0-py2.py3-none-any.whl (9.0 kB)
  Using cached subprocess32-3.5.4-py3-none-any.whl
  Using cached pathtools-0.1.2-py3-none-any.whl
  Using cached shortuuid-1.0.1-py3-none-any.whl (7.5 kB)
  Using cached gitdb-4.0.7-py3-none-any.whl (63 kB)
  Using cached smmap-4.0.0-py2.py3-none-any.whl (24 kB)
  Using cached xxhash-2.0.2-cp37-cp

## Data Loading and Exploration

In [7]:
import pandas as pd

In [8]:
from datasets import load_dataset
#from datasets import list_datasets

In [9]:
#Download hope speech dataset in english.
dataset= load_dataset("hope_edi", "english")

Downloading:   0%|          | 0.00/2.90k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.34k [00:00<?, ?B/s]

Downloading: 0.00B [00:00, ?B/s]

Downloading:   0%|          | 0.00/305k [00:00<?, ?B/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset hope_edi downloaded and prepared to /root/.cache/huggingface/datasets/hope_edi/english/1.0.0/fff5cf6e767fe3d1de7c5df863565bdce10bfe79dfb0b2ce42d320c3864497e3. Subsequent calls will reuse this data.


Create a datsa DataFrame for the Train, validation and test datasets

In [10]:
from pandas import DataFrame

In [11]:
eng_train=DataFrame({'text':dataset['train']['text'], 'label': dataset['train']['label']})
eng_val=DataFrame({'text':dataset['validation']['text'], 'label': dataset['validation']['label']})

#Use train test split to split 20% of the train dataset into a test dataset
from sklearn.model_selection import train_test_split
eng_train, eng_test = train_test_split(eng_train, test_size=0.2)


### Data Expolration

In [12]:
eng_train['label'].value_counts()

1    16644
0     1547
2       18
Name: label, dtype: int64

In [13]:
# Remove label 2 from datset. This will leave just hope and not hope labels: 0 and 1
eng_train=eng_train.drop(eng_train.query('label == 2').sample(frac=1).index)
eng_train=eng_train.drop(eng_train.query('label == 1').sample(frac=0.3).index)
eng_test=eng_test.drop(eng_test.query('label == 2').sample(frac=1).index)
eng_val=eng_val.drop(eng_val.query('label == 2').sample(frac=1).index)

Get a count of each label for each dataset

In [14]:
eng_train['label'].value_counts()


1    11651
0     1547
Name: label, dtype: int64

In [15]:
eng_test['label'].value_counts()


1    4134
0     415
Name: label, dtype: int64

In [16]:

eng_val['label'].value_counts()

1    2569
0     272
Name: label, dtype: int64

From the counts of each label, there seems to be a clear imbalance in the weights.  This could affect the performace of the model.  This will be dealt with at a later stage in the code

In [17]:
df_train= eng_train
df_dev= eng_val
df_test = eng_test

## Building the model

In [21]:
#import GPUtil as GPU
import numpy as np
import pandas as pd
from tqdm import tqdm
from scipy.special import softmax
from simpletransformers.classification import ClassificationModel
from sklearn.model_selection import train_test_split, StratifiedKFold, KFold 
import sklearn
from sklearn.metrics import log_loss
from sklearn.metrics import *
from sklearn.model_selection import *
import torch


In [22]:
labels = list(df_train['label'].unique())
labels

[1, 0]

In [23]:

df_train ['label'] = df_train['label'].astype('category')
df_dev ['label'] = df_dev['label'].astype('category')
df_test['label'] = df_test['label'].astype('category')

In [24]:
labels_dic = dict(enumerate(df_train['label'].cat.categories))

In [25]:
df_train['label'] = df_train['label'].cat.codes
df_dev['label'] = df_dev['label'].cat.codes

In [26]:
df_train.head()

,text,label
9897,Every life matter!! Why u boo him? The devil!,1
2615,These people don't want justice,1
17070,To be honest the beginning made my feel worse ...,1
11343,My god says don't then I have to do the order,1
17780,@Sandy Castles In a indirect way your right. D...,1


In [27]:
df_train.sample(30)

,text,label
21627,@Kiwi what about the white guy that had the sa...,1
77,She saves lives with her music.,0
2544,Y’all need to listen to this,1
10580,Truth be told Native Indians are the most forg...,1
19322,if this comment is anything to go by yeagh rac...,1
9386,Madonna the QUEEN. Intemporel,1
5153,The fact we are being told it’s wrong to tell ...,1
2478,If sexuality was a choice,1
11778,Sheriff Clark is correct,1
257,@Rick Gross you realize that simply not liking...,1


In [28]:
labels_dic

{0: 0, 1: 1}

In [29]:
np.unique(df_train['label'])

array([0, 1], dtype=int8)

In [30]:
# This code will deal with the weight imbalance. This should improve the performace of the model
from sklearn.utils import class_weight
import numpy as np
class_weights = class_weight.compute_class_weight('balanced',
                                                 np.unique(df_train['label']),
                                                 df_train['label'])
class_weights

array([4.2656755 , 0.56638915])

In [31]:
sweep_config = {
    
    "method": "bayes",  # grid, random
    "metric": {"name": "eval_loss", "goal": "minimize"},
    "parameters": {
        "num_train_epochs": {"values": [2, 3, 4, 5, 6]},
        "learning_rate": {"min": 5e-6, "max": 4e-3},
        'weight_decay':  {"values": [0, 0.1, 0.01, 0.001]}
    },
}
SEED= 2021

In [32]:
import logging
import wandb

In [33]:
logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.INFO)

In [34]:
sweep_id = wandb.sweep(sweep_config, project="Model 4 Sweeps")

<IPython.core.display.Javascript object>

wandb: You can find your API key in your browser here: https://wandb.ai/authorize


wandb: Paste an API key from your profile and hit enter: ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Create sweep with ID: z69bz2fd
Sweep URL: https://wandb.ai/dabbott93/Model%204%20Sweeps/sweeps/z69bz2fd


In [35]:
import torch
# Check if there's a GPU available...
if torch.cuda.is_available():    
    # If a GPU is available tell PyTorch to use the GPU.    
    device = torch.device("cuda")
    # Print that a GPU is available and its name
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
# If a GPU is not available print the following statement
else:
    print('No GPU available, using the CPU instead.')

There are 1 GPU(s) available.
We will use the GPU: Tesla T4


###Hyperparameter Optimisation using Weights and Biases
This code will run through a series of sweeps to find the best combination of hyperparameters.

In [36]:
import numpy as np

n=5
kf = KFold(n_splits=n, random_state=SEED, shuffle=True)

def train():
  #Initialize a new wandb run
  %%time
  err=[]
  y_pred_tot=[]
  #Initialize a new wandb run
  wandb.init()
  i=1
  

  model = ClassificationModel('bert', 'bert-base-uncased', use_cuda=True, num_labels=2, args={
                                                                      'train_batch_size':16,
                                                                      'reprocess_input_data': True,
                                                                      "weight":  class_weights,
                                                                      'overwrite_output_dir': True,#Overwrite the content of the output directory. I set it to True in case I run the notebook multiple times I only care about the last run.
                                                                      'fp16': False,
                                                                      'do_lower_case': False,
                                                                      'num_train_epochs': 4,
                                                                      'max_seq_length': 256,
                                                                      'regression': False,
                                                                      'manual_seed': SEED,
                                                                      "learning_rate":2e-5,
                                                                      'weight_decay':0,
                                                                      "save_eval_checkpoints": False,
                                                                      "save_model_every_epoch": False,
                                                                      "silent": False,
                                                                        "verbose": True,
                                                                      "evaluate_during_training": False,
                                                                      "use_early_stopping":True,
                                                                      "dataloader_num_workers": 0,
                                                                      'use_multiprocessing': False,
                                                                      "output_dir":'outputs/'+'eng'+'/',
                                                                      "wandb_project": "Model 4 Sweeps"}, 
                                                                      sweep_config = wandb.config,
  )                                                                  



  results = []
  TP= []
  TN=[]
  FP=[]
  FN=[]

  
  model.train_model(df_train)
  # validate the model 
  result, model_outputs, wrong_predictions = model.eval_model(df_dev, acc=accuracy_score)
  print(result['acc'])
  # append model score
  results.append(result['acc'])
  TP.append(result['tp'])
  TN.append(result['tn'])
  FP.append(result['fp'])
  FN.append(result['fn'])

  #PRINT WEIGHTED F1 TABLE
  print('WEIGHTED AVERAGES')
  raw_outputs_vals = softmax(model_outputs,axis=1)
  y_pred = [np.argmax(i) for i in  raw_outputs_vals]
  sklearn.metrics.f1_score(df_dev['label'].to_numpy(),y_pred,average = 'micro')
  print(sklearn.metrics.classification_report(df_dev['label'].to_numpy(),y_pred))

  #Sync wandb
  wandb.join()

In [37]:
import time
training_start_time= time.time()

In [38]:
wandb.agent(sweep_id, train)

INFO:wandb.agents.pyagent:Starting sweep agent: entity=None, project=None, count=None
wandb: Agent Starting Run: c4ck1gtx with config:
wandb: 	learning_rate: 0.0014743179067724435
wandb: 	num_train_epochs: 3
wandb: 	weight_decay: 0.001


CPU times: user 3 µs, sys: 1e+03 ns, total: 4 µs
Wall time: 7.15 µs


wandb: Currently logged in as: dabbott93 (use `wandb login --relogin` to force relogin)


INFO:filelock:Lock 140388912840272 acquired on /root/.cache/huggingface/transformers/3c61d016573b14f7f008c02c4e51a366c67ab274726fe2910691e2a761acf43e.37395cee442ab11005bcd270f3c34464dc1704b715b5d7d52b1a461abe3b9e4e.lock
https://huggingface.co/bert-base-uncased/resolve/main/config.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmpe1vq8qto


Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

...

Epoch:   0%|          | 0/6 [00:00<?, ?it/s]

Running Epoch 0 of 6:   0%|          | 0/825 [00:00<?, ?it/s]

Running Epoch 1 of 6:   0%|          | 0/825 [00:00<?, ?it/s]

Running Epoch 2 of 6:   0%|          | 0/825 [00:00<?, ?it/s]

Configuration saved in outputs/eng/checkpoint-2000/config.json
Model weights saved in outputs/eng/checkpoint-2000/pytorch_model.bin
tokenizer config file saved in outputs/eng/checkpoint-2000/tokenizer_config.json
Special tokens file saved in outputs/eng/checkpoint-2000/special_tokens_map.json
wandb: Ctrl + C detected. Stopping sweep.


In [39]:
print('training finished, took{:.2f}s'.format(time.time()-training_start_time))

training finished, took4574.80s


## K-fold cross validation
This code will split the traning dataset into 5 folds.
It will train the model with the most optiised combination of parameter that  calculated previously.

In [40]:
import time
training_start_time= time.time()

In [41]:
import numpy as np
train_data= df_train
n=5
kf = KFold(n_splits=n, random_state=SEED, shuffle=True)
wandb.init()

# %%time
#  err=[]
#  y_pred_tot=[]

#  wandb.init()

#  i=1

for train_index, val_index in kf.split(train_data):
    results = []
    TP= []
    TN=[]
    FP=[]
    FN=[]

    #splitting Dataframe (dataset not included)
    train_df = train_data.iloc[train_index]
    val_df = train_data.iloc[val_index]
    # Defining Model
    
    model = ClassificationModel('bert', 'bert-base-uncased', use_cuda=True, num_labels=2, args={
                                                                      'train_batch_size':16,
                                                                      'reprocess_input_data': True,
                                                                      "weight":  class_weights,
                                                                      'overwrite_output_dir': True,#Overwrite the content of the output directory. I set it to True in case I run the notebook multiple times I only care about the last run.
                                                                      'fp16': False,
                                                                      'do_lower_case': False,
                                                                      'num_train_epochs': 6,
                                                                      'max_seq_length': 256,
                                                                      'regression': False,
                                                                      'manual_seed': SEED,
                                                                      "learning_rate":3e-5,
                                                                      'weight_decay':0.001,
                                                                      "save_eval_checkpoints": False,
                                                                      "save_model_every_epoch": False,
                                                                      "silent": False,
                                                                        "verbose": True,
                                                                      "evaluate_during_training": False,
                                                                      "use_early_stopping":True,
                                                                      "dataloader_num_workers": 0,
                                                                      'use_multiprocessing': False,
                                                                      "output_dir":'outputs/'+'eng'+'/', 
                                                                      "wandb_project": "Model 4 K-fold Cross val"} 
  )
    # train the model
    model.train_model(train_df)
    # validate the model 
    result, model_outputs, wrong_predictions = model.eval_model(val_df, acc=accuracy_score)
    print(result['acc'])
    # append model score
    results.append(result['acc'])
    TP.append(result['tp'])
    TN.append(result['tn'])
    FP.append(result['fp'])
    FN.append(result['fn'])
    print('TP, TN, FP, FN', TP, TN, FP, FN)

    #PRINT WEIGHTED F1 TABLE
    print('WEIGHTED AVERAGES')
    raw_outputs_vals = softmax(model_outputs,axis=1)
    y_pred = [np.argmax(i) for i in  raw_outputs_vals]
    sklearn.metrics.f1_score(val_df['label'].to_numpy(),y_pred,average = 'micro')
    print(sklearn.metrics.classification_report(val_df['label'].to_numpy(),y_pred))

Training loss,0.27043
lr,0.00205
global_step,2050
_runtime,1466
_timestamp,1630605780
_step,40


Training loss,▂▃▃▃█▃▄▃▃▂▂▃▃▂▄▂▂▁▃▃▂▃▄▂▁▂▂▂▂▄▃▂▂▃▃▂▃▂▃▂
lr,▁▂▄▅▇██████▇▇▇▇▇▇▇▇▇▇▇▆▆▆▆▆▆▆▆▆▆▆▅▅▅▅▅▅▅
global_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇██
_runtime,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇██
_timestamp,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇██
_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇██


loading configuration file https://huggingface.co/bert-base-uncased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/3c61d016573b14f7f008c02c4e51a366c67ab274726fe2910691e2a761acf43e.37395cee442ab11005bcd270f3c34464dc1704b715b5d7d52b1a461abe3b9e4e
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.10.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights file https://huggingface.co/bert-base-uncased/resolve/main/pytorch_model.bin from cache 

Epoch:   0%|          | 0/6 [00:00<?, ?it/s]

Running Epoch 0 of 6:   0%|          | 0/660 [00:00<?, ?it/s]

...

  0%|          | 0/2639 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_utils: Saving features into cached file cache_dir/cached_dev_bert_256_2_2


Running Evaluation:   0%|          | 0/330 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_model:{'mcc': 0.4910949531434918, 'tp': 2200, 'tn': 166, 'fp': 150, 'fn': 123, 'auroc': 0.828669142368282, 'auprc': 0.9647522305555398, 'acc': 0.896551724137931, 'eval_loss': 0.6441181685059167}


0.896551724137931
TP, TN, FP, FN [2200] [166] [150] [123]
WEIGHTED AVERAGES
              precision    recall  f1-score   support

           0       0.57      0.53      0.55       316
           1       0.94      0.95      0.94      2323

    accuracy                           0.90      2639
   macro avg       0.76      0.74      0.75      2639
weighted avg       0.89      0.90      0.89      2639



In [42]:
print('training finished, took{:.2f}s'.format(time.time()-training_start_time))

training finished, took14339.24s


## Final trained model
This model is traning with the most optimised hyperparameters

In [43]:
%%time
err=[]
y_pred_tot=[]
wandb.init()

model = ClassificationModel('bert', 'bert-base-uncased', use_cuda=True, num_labels=2, args={
                                                                     'train_batch_size':16,
                                                                     'reprocess_input_data': True,
                                                                     "weight":  [1,0.1],
                                                                     'overwrite_output_dir': True,#Overwrite the content of the output directory. I set it to True in case I run the notebook multiple times I only care about the last run.
                                                                     'fp16': False,
                                                                     'do_lower_case': False,
                                                                     'num_train_epochs': 6,
                                                                     'max_seq_length': 256,
                                                                     'regression': False,
                                                                     'manual_seed': SEED,
                                                                     "learning_rate":3e-5,
                                                                     'weight_decay':0.001,
                                                                     "save_eval_checkpoints": False,
                                                                     "save_model_every_epoch": False,
                                                                     "silent": False,
                                                                      "verbose": True,
                                                                    "evaluate_during_training": True,
                                                                    "use_early_stopping":True,
                                                                    "dataloader_num_workers": 0,
                                                                    'use_multiprocessing': False,
                                                                    "output_dir":'outputs/'+'eng'+'/',
                                                                    "wandb_project": "Model 4 final ",                                                                    
                                                                    "early_stopping_delta" : 0.01,
                                                                    "early_stopping_metric" : "mcc",
                                                                    "early_stopping_metric_minimize" : False,
                                                                    "early_stopping_patience" : 5,
                                                                    "evaluate_during_training_steps" : 1000} 
)                                                                  



Training loss,0.00392
lr,0.0
global_step,3950
_runtime,14339
_timestamp,1630620140
_step,409


Training loss,▅▁▅▁▁▁▁▁▇▂▃▁▁▁▁▃█▂▅▆▁▁▁▁▇▄▁▁▁▁▇▁▃▃▃█▁▅▁▁
lr,▇█▇▆▅▄▃▁▂█▇▆▅▄▃▁▄█▇▆▅▃▃▁▆█▇▅▅▃▂▁▇█▇▅▄▃▂▁
global_step,▁▂▃▄▅▆▇█▁▂▃▄▅▆▇█▁▂▃▄▅▆▇█▁▂▃▄▅▆▇█▁▂▃▄▅▆▇█
_runtime,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
_timestamp,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███


loading configuration file https://huggingface.co/bert-base-uncased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/3c61d016573b14f7f008c02c4e51a366c67ab274726fe2910691e2a761acf43e.37395cee442ab11005bcd270f3c34464dc1704b715b5d7d52b1a461abe3b9e4e
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.10.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights file https://huggingface.co/bert-base-uncased/resolve/main/pytorch_model.bin from cache 

CPU times: user 1.35 s, sys: 526 ms, total: 1.88 s
Wall time: 17.8 s


In [44]:
training_start_time= time.time()

In [45]:
model.train_model(df_train, eval_df = df_dev)

/usr/local/lib/python3.7/dist-packages/simpletransformers/classification/classification_model.py:616: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."
INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.
INFO:simpletransformers.classification.classification_utils: Saving features into cached file cache_dir/cached_train_bert_256_2_2


Epoch:   0%|          | 0/6 [00:00<?, ?it/s]

Running Epoch 0 of 6:   0%|          | 0/825 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/simpletransformers/classification/classification_model.py:1402: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."
INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.


  0%|          | 0/2841 [00:00<?, ?it/s]

...

Running Epoch 5 of 6:   0%|          | 0/825 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.


  0%|          | 0/2841 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_utils: Saving features into cached file cache_dir/cached_dev_bert_256_2_2
Configuration saved in outputs/eng/config.json
Model weights saved in outputs/eng/pytorch_model.bin
tokenizer config file saved in outputs/eng/tokenizer_config.json
Special tokens file saved in outputs/eng/special_tokens_map.json
INFO:simpletransformers.classification.classification_model: Training of bert model complete. Saved to outputs/eng/.


(4950,
 {'auprc': [0.9837315984270888,
   0.9831426049518429,
   0.9864097882135476,
   0.9850093797576023,
   0.9829696963206838,
   0.9816353066009469,
   0.9833809382556906,
   0.9795800588777811,
   0.976684464122985,
   0.975696985922023],
  'auroc': [0.8788732454834796,
   0.8728383383326084,
   0.8955754985918073,
   0.8885803299521444,
   0.8820159194467978,
   0.8740776624001099,
   0.877218905273281,
   0.8733936013097336,
   0.8614110835069723,
   0.8660234870514963],
  'eval_loss': [0.2319671766978971,
   0.24637978956014378,
   0.2389283853719074,
   0.29868084769185865,
   0.3086509636176222,
   0.4056749769273604,
   0.4156398557373741,
   0.517842560012356,
   0.5474128321886914,
   0.5592253388235855],
  'fn': [83, 90, 154, 122, 116, 124, 136, 145, 154, 156],
  'fp': [166, 147, 114, 134, 135, 142, 139, 132, 124, 126],
  'global_step': [825, 1000, 1650, 2000, 2475, 3000, 3300, 4000, 4125, 4950],
  'mcc': [0.42198835185043404,
   0.4722847664676089,
   0.4902299846682451

In [46]:
print('training finished, took{:.2f}s'.format(time.time()-training_start_time))

training finished, took4040.77s


In [47]:
raw_outputs_vals=[]

In [48]:
raw_outputs_vals

[]

In [49]:
result, model_outputs, wrong_predictions = model.eval_model(df_dev)
print(result)


/usr/local/lib/python3.7/dist-packages/simpletransformers/classification/classification_model.py:1402: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."
INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.


  0%|          | 0/2841 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_utils: Saving features into cached file cache_dir/cached_dev_bert_256_2_2


Running Evaluation:   0%|          | 0/356 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_model:{'mcc': 0.4544391691106145, 'tp': 2413, 'tn': 146, 'fp': 126, 'fn': 156, 'auroc': 0.8660234870514963, 'auprc': 0.975696985922023, 'eval_loss': 0.5592253388235855}


{'mcc': 0.4544391691106145, 'tp': 2413, 'tn': 146, 'fp': 126, 'fn': 156, 'auroc': 0.8660234870514963, 'auprc': 0.975696985922023, 'eval_loss': 0.5592253388235855}


In [50]:
raw_outputs_vals = softmax(model_outputs,axis=1)

In [51]:
raw_outputs_vals

array([[1.81085409e-04, 9.99818915e-01],
       [2.15316734e-04, 9.99784683e-01],
       [3.70335649e-03, 9.96296644e-01],
       ...,
       [2.22935031e-04, 9.99777065e-01],
       [1.84661753e-04, 9.99815338e-01],
       [2.87554241e-04, 9.99712446e-01]])

In [52]:
print(f"Log_Loss: {log_loss(df_dev['label'], raw_outputs_vals)}")

Log_Loss: 0.5405121718599079


In [53]:
##import numpy as np
y_pred = [np.argmax(i) for i in  raw_outputs_vals]

In [54]:
sklearn.metrics.f1_score(df_dev['label'].to_numpy(),y_pred,average = 'micro')

0.9007391763463569

In [55]:
print(sklearn.metrics.classification_report(df_dev['label'].to_numpy(),y_pred))

              precision    recall  f1-score   support

           0       0.48      0.54      0.51       272
           1       0.95      0.94      0.94      2569

    accuracy                           0.90      2841
   macro avg       0.72      0.74      0.73      2841
weighted avg       0.91      0.90      0.90      2841



### Test Set Predictions

In [56]:
import torch

In [57]:

model = ClassificationModel('bert', 'outputs/'+'eng'+'/')


loading configuration file outputs/eng/config.json
Model config BertConfig {
  "_name_or_path": "bert-base-uncased",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "torch_dtype": "float32",
  "transformers_version": "4.10.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights file outputs/eng/pytorch_model.bin
All model checkpoint weights were used when initializing BertForSequenceClassification.

All the weights of BertForSequenceClassification were initialized from the model checkpoint at outputs/eng

In [58]:
test_sents = list(df_test['text'])

In [59]:
predictions, raw_outputs = model.predict(test_sents)

INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.


  0%|          | 0/4549 [00:00<?, ?it/s]

  0%|          | 0/569 [00:00<?, ?it/s]

In [60]:
labels_dic

{0: 0, 1: 1}

In [61]:
predictions = [labels_dic[i] for i in predictions]
#predictions

In [62]:
df_test['label_pred'] = predictions

In [63]:
df_test.head()

,text,label,label_pred
4834,@'` exactly,1,1
21996,@Hippo Bottom Jeans Where? I didn't care that ...,1,1
20137,How is she out of breath sitting down?,1,1
12857,Just because someone has short hair doesn’t me...,1,1
14619,May I please get a #BlackLivesMatter in the re...,1,1


In [64]:
print(sklearn.metrics.classification_report(df_test['label'].to_numpy(),predictions))

              precision    recall  f1-score   support

           0       0.48      0.54      0.51       415
           1       0.95      0.94      0.95      4134

    accuracy                           0.90      4549
   macro avg       0.72      0.74      0.73      4549
weighted avg       0.91      0.90      0.91      4549



In [65]:
lang= 'eng'

In [66]:
df_test['correct'] = df_test.apply(lambda row: row['label'] == row ['label_pred'], axis = 1)
ids = [lang +'_sent_'+str(i) for i in range(1,len(df_test)+1)]
df_test['id'] = ids
df_test = df_test.set_index('id')
df_test.head()
#df_test= df_test.drop(columns=['labels'])

,text,label,label_pred,correct
id,,,,
eng_sent_1,@'` exactly,1,1,True
eng_sent_2,@Hippo Bottom Jeans Where? I didn't care that ...,1,1,True
eng_sent_3,How is she out of breath sitting down?,1,1,True
eng_sent_4,Just because someone has short hair doesn’t me...,1,1,True
eng_sent_5,May I please get a #BlackLivesMatter in the re...,1,1,True


In [67]:
df_test.to_csv('DeanAbbott_Model4.tsv',sep='\t')



In [68]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

In [69]:
cm=sklearn.metrics.confusion_matrix(df_test['label'].to_numpy(),predictions)

In [70]:
cm

array([[ 223,  192],
       [ 242, 3892]])

#Testing on other datasets -Domain Transfer/Transfer learning/Generalisation

#Covid dataset


In [71]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [72]:
df2 = pd.read_csv("/content/drive/MyDrive/covid dataset.csv",encoding='latin1')
df2.head()

,UserName,ScreenName,Location,TweetAt,OriginalTweet,Sentiment
0,3799,48751,London,16/03/2020,@MeNyrbie @Phil_Gahan @Chrisitv https://t.co/i...,Neutral
1,3800,48752,UK,16/03/2020,advice Talk to your neighbours family to excha...,Positive
2,3801,48753,Vagabonds,16/03/2020,Coronavirus Australia: Woolworths to give elde...,Positive
3,3802,48754,NaN,16/03/2020,My food stock is not the only one which is emp...,Positive
4,3803,48755,NaN,16/03/2020,"Me, ready to go at supermarket during the #COV...",Extremely Negative


In [73]:
df2['Sentiment'] = df2['Sentiment'].map({'Positive':int(1), 'Extremely Positive':int(0),'Negative':int(1),'Extremely Negative':int(1),'Neutral':int(1)})
df2['Sentiment'] = df2['Sentiment'].astype('category')

df2.head()

,UserName,ScreenName,Location,TweetAt,OriginalTweet,Sentiment
0,3799,48751,London,16/03/2020,@MeNyrbie @Phil_Gahan @Chrisitv https://t.co/i...,1.0
1,3800,48752,UK,16/03/2020,advice Talk to your neighbours family to excha...,1.0
2,3801,48753,Vagabonds,16/03/2020,Coronavirus Australia: Woolworths to give elde...,1.0
3,3802,48754,NaN,16/03/2020,My food stock is not the only one which is emp...,1.0
4,3803,48755,NaN,16/03/2020,"Me, ready to go at supermarket during the #COV...",1.0


In [74]:
df2.dropna(subset = ["Sentiment"], inplace=True)

In [75]:
df2.head()

,UserName,ScreenName,Location,TweetAt,OriginalTweet,Sentiment
0,3799,48751,London,16/03/2020,@MeNyrbie @Phil_Gahan @Chrisitv https://t.co/i...,1.0
1,3800,48752,UK,16/03/2020,advice Talk to your neighbours family to excha...,1.0
2,3801,48753,Vagabonds,16/03/2020,Coronavirus Australia: Woolworths to give elde...,1.0
3,3802,48754,NaN,16/03/2020,My food stock is not the only one which is emp...,1.0
4,3803,48755,NaN,16/03/2020,"Me, ready to go at supermarket during the #COV...",1.0


In [76]:
df2['Sentiment'].value_counts()

1.0    6842
0.0    1212
Name: Sentiment, dtype: int64

In [77]:
#df2=df2.drop(df2.query('Sentiment == 1').sample(frac=0.5).index)

In [78]:
df2['Sentiment'].value_counts()

1.0    6842
0.0    1212
Name: Sentiment, dtype: int64

In [79]:
test_sents = list(df2['OriginalTweet'])

In [80]:
predictions, raw_outputs = model.predict(test_sents)

INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.


  0%|          | 0/8054 [00:00<?, ?it/s]

  0%|          | 0/1007 [00:00<?, ?it/s]

In [81]:
labels_dic_test = dict(enumerate(df2['Sentiment'].cat.categories))

In [82]:
predictions = [labels_dic_test[i] for i in predictions]

In [83]:
df2['label_pred'] = predictions

In [84]:
df2['label_pred'].value_counts()

1.0    7929
0.0     125
Name: label_pred, dtype: int64

In [85]:
df2['label_pred'].value_counts()

1.0    7929
0.0     125
Name: label_pred, dtype: int64

In [86]:
df2.head()

,UserName,ScreenName,Location,TweetAt,OriginalTweet,Sentiment,label_pred
0,3799,48751,London,16/03/2020,@MeNyrbie @Phil_Gahan @Chrisitv https://t.co/i...,1.0,1.0
1,3800,48752,UK,16/03/2020,advice Talk to your neighbours family to excha...,1.0,1.0
2,3801,48753,Vagabonds,16/03/2020,Coronavirus Australia: Woolworths to give elde...,1.0,1.0
3,3802,48754,NaN,16/03/2020,My food stock is not the only one which is emp...,1.0,1.0
4,3803,48755,NaN,16/03/2020,"Me, ready to go at supermarket during the #COV...",1.0,1.0


In [87]:
print(sklearn.metrics.classification_report(df2['Sentiment'].to_numpy(),predictions))

              precision    recall  f1-score   support

         0.0       0.43      0.04      0.08      1212
         1.0       0.85      0.99      0.92      6842

    accuracy                           0.85      8054
   macro avg       0.64      0.52      0.50      8054
weighted avg       0.79      0.85      0.79      8054



#Airline tweets

In [88]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [89]:
df3 = pd.read_csv("/content/drive/MyDrive/Tweets.csv",encoding='latin1')
df3.head()

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,570306133677760513,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada)
1,570301130888122368,positive,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada)
2,570301083672813571,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I n...,NaN,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada)
3,570301031407624196,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica it's really aggressive to blast...,NaN,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada)
4,570300817074462722,negative,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing...,NaN,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada)


In [90]:
df3['airline_sentiment'].value_counts()

negative    9178
neutral     3099
positive    2363
Name: airline_sentiment, dtype: int64

In [91]:
df3['airline_sentiment'] = df3['airline_sentiment'].map({'positive':int(0),'negative':int(1),'neutral':int(1)})
df3['airline_sentiment'] = df3['airline_sentiment'].astype('category')

df3.head()

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,570306133677760513,1,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada)
1,570301130888122368,0,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada)
2,570301083672813571,1,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I n...,NaN,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada)
3,570301031407624196,1,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica it's really aggressive to blast...,NaN,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada)
4,570300817074462722,1,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing...,NaN,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada)


In [92]:
df3.dropna(subset = ["airline_sentiment"], inplace=True)

In [93]:
df3.head()

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,570306133677760513,1,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada)
1,570301130888122368,0,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada)
2,570301083672813571,1,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I n...,NaN,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada)
3,570301031407624196,1,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica it's really aggressive to blast...,NaN,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada)
4,570300817074462722,1,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing...,NaN,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada)


In [94]:
df3['airline_sentiment'].value_counts()

1    12277
0     2363
Name: airline_sentiment, dtype: int64

In [95]:
# df3=df3.drop(df3.query('airline_sentiment == 1').sample(frac=0.5).index)

In [96]:
test_sents = list(df3['text'])

In [97]:
df3['airline_sentiment'].value_counts()

1    12277
0     2363
Name: airline_sentiment, dtype: int64

In [98]:
predictions2, raw_outputs = model.predict(test_sents)

INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.


  0%|          | 0/14640 [00:00<?, ?it/s]

  0%|          | 0/1830 [00:00<?, ?it/s]

In [99]:
labels_dic_test = dict(enumerate(df3['airline_sentiment'].cat.categories))

In [100]:
predictions2 = [labels_dic_test[i] for i in predictions2]

In [101]:
df3['label_pred'] = predictions2

In [102]:
df3['label_pred'].value_counts()

1    14521
0      119
Name: label_pred, dtype: int64

In [103]:
df3.head()

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone,label_pred
0,570306133677760513,1,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada),1
1,570301130888122368,0,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada),1
2,570301083672813571,1,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I n...,NaN,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada),1
3,570301031407624196,1,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica it's really aggressive to blast...,NaN,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada),1
4,570300817074462722,1,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing...,NaN,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada),1


In [104]:
print(sklearn.metrics.classification_report(df3['airline_sentiment'].to_numpy(),predictions2))

              precision    recall  f1-score   support

           0       0.66      0.03      0.06      2363
           1       0.84      1.00      0.91     12277

    accuracy                           0.84     14640
   macro avg       0.75      0.52      0.49     14640
weighted avg       0.81      0.84      0.78     14640



In [105]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

In [106]:
disp = ConfusionMatrixDisplay(confusion_matrix=cm,display_labels=df3['airline_sentiment'].to_numpy())